# Imports

In [1]:
import dlt
import requests
import pandas as pd
from datetime import datetime

# Q1

In [2]:
# What's the version of dlt that you installed?

In [3]:
!dlt --version

dlt 1.14.1


# Q2

In [2]:
from dlt.destinations import qdrant

In [4]:
# dir(qdrant)

In [ ]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

from dlt.destinations.adapters import qdrant_adapter

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)

# load_info = pipeline.run(zoomcamp_data())
# print(pipeline.last_trace)

load_info = pipeline.run(
    qdrant_adapter(zoomcamp_data(), embed=["text", "section", "question"])
)
print(pipeline.last_trace)

dataset = pipeline.dataset().zoomcamp_data.df()

In [5]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [6]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [7]:
from dlt.destinations.adapters import qdrant_adapter

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)

# load_info = pipeline.run(zoomcamp_data())
# print(pipeline.last_trace)

load_info = pipeline.run(
    qdrant_adapter(zoomcamp_data(), embed=["text", "section", "question"])
)
print(pipeline.last_trace)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Run started at 2025-07-19 20:02:28.289581+00:00 and COMPLETED in 3 minutes and 57.81 seconds with 4 steps.
Step extract COMPLETED in 0.55 seconds.

Load package 1752955349.870657 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.07 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)

Load package 1752955349.870657 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 3 minutes and 55.62 seconds.
Pipeline zoomcamp_pipeline load step completed in 3 minutes and 55.60 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/0b-dlt/db.qdrant location to store data
Load package 1752955349.870657 is LOADED and contains no failed jobs

Step run COMPLETED in 3 minutes and 57.80 seconds.
Pipeline zoomcamp_pipeline load step completed in 3 minutes and 55.60 seconds
1 lo

In [27]:
# How many rows were inserted into the zoomcamp_data collection?
from qdrant_client import QdrantClient

client = QdrantClient(path="db.qdrant") 
client.get_collection("zoomcamp_tagged_data_zoomcamp_data").points_count

948

# Q3

In [ ]:
# When inserting the data, an embedding model was used. Which one?
fast-bge-small-en